In [3]:
import pandas as pd
from datetime import datetime
import logging

# Load Data

In [4]:
nba       = pd.read_csv('../data/nba/nba_salaries.csv')
insurance = pd.read_csv('../data/insurance/insurance.csv')
airline   = pd.read_csv('../data/airline/train.csv')
airbnb    = pd.read_csv('../data/airbnb/listings.csv')

# Basic Data Wrangling


Function to handle the following issues:

1. Empty columns;
2. Categorical columns where the majority of values are unique
3. Columns with only one unique value

In [5]:
def basic_wrangling(df: pd.DataFrame, columns : list = None, 
                    proportion_nan_thresh : float = 0.95, 
                    proprotion_unique_thresh : float = 0.95):
    '''
    Function to execute some basic data wrangling in pandas dataframe. 
    The following operations will be executed:
        
        1. Remove empty or almost empty columns 
        2. Remove categorical columns with more than proprotion_unique_thresh 
            unique values
        3. Remove columns composed by a single value

    Args:
        df - pandas Dataframe
        columns - list of columns to be wrngled. If None, operations will 
                  be applied to all columns in dataframe
        proportion_nan_thresh - If the proportion of NAN values in a column 
                    is greater than proportion_nan_thresh the column will be 
                    removed. Default = 0.95
        proportion_unique_thresh - If the proportion of unique values in a 
                    categorical column is greater than proportion_unique_thresh 
                    the column will be removed. Default = 0.95

    Returns:
        df - wrangled pandas dataframe
    '''

    df = df.copy()

    if columns is None:
        columns = df.columns

    n_rows = len(df) #number of rows

    for feature in columns:
        if feature in df.columns:
            #remove empty or almost empy columns
            num_empty_values = df[feature].isna().sum()
            proportion_nan_values = num_empty_values / n_rows
            num_unique = df[feature].nunique()
            proportion_unique_values = num_unique / n_rows
            data_type = df[feature].dtype
            if proportion_nan_values > proportion_nan_thresh:
                df = df.drop(columns = feature)
                print(
                f'Feature {feature} removed. '
                f'{100 * proportion_nan_values:.2f}% of missing values.'
                )

            #remove features with only one unique value
            elif num_unique == 1:
                df = df.drop(columns = feature)
                print(f'Feature {feature} removed. ' 
                      'There is only one unique value in the column.')
                
            #Remove categorical columns with more than proprotion_unique_thresh 
            #unique values
            elif (proportion_unique_values > proprotion_unique_thresh 
                  and not pd.api.types.is_numeric_dtype(df[feature])):
                df= df.drop(columns = feature)
                print(f'Feature {feature} removed. '
                      f'The proportion of unique values in the feature of '
                      f'type  {data_type} is ' 
                      f'{100 * proportion_unique_values:.2f}%')

        else:
            print(f'The feature {feature} is not in the dataframe.')

    

In [6]:
def parse_and_format_dates(date_string: str, 
                           standard_format: str = "%Y-%m-%d", 
                           return_type: str = 'string'):
    """
    Convert a date string into a datetime object or a formatted string.

    The function tests if the input string matches any of the following 
    date formats:
    
        "%Y-%m-%d",
        "%d-%m-%Y",
        "%m/%d/%Y",
        "%d %b %Y",
        "%B %d, %Y"

    If the input string doesn't follow any of these formats, the function will 
    return None.
    It can be returned a datetime object or a string. 
        
    Args:
        date_string: date in string format
        standard_format: format of data that will be returned
        return_type: Type of object to return, either 'datetime' or 'string'.

    Returns:
        Union[datetime, str, None]: Parsed date as a datetime object or a 
        formatted string. Returns None if parsing fails.

    """

    #accepted
    formats = [ "%Y-%m-%d",
                "%d-%m-%Y",
                "%m/%d/%Y",
                "%d %b %Y",
                "%B %d, %Y"]
    
    #@if the input is not a string, return None
    if not isinstance(date_string, str):
        return None
    if return_type not in ('string', 'datetime'):
        print (f'{return_type} is not known. The formats accepted are string '
               'and datetime. String will be used as default')
        return_type = 'string'
        
    
    for expected_format in formats:
        try:
            #convert to datetime using the format specified in expected_format
            parsed_date = datetime.strptime(date_string, expected_format) 
            if return_type == 'datetime':
                return parsed_date #return datatime object
            else:
                return parsed_date.strftime(standard_format) #return string 
        except ValueError:
            continue
    
    return None
    


In [7]:
def create_new_date_columns(df, features_list: list, 
                            calculate_difference: bool = True,
                            reference_date: str = None):
    """
    Create new columns with extracted date parts and (optionally) date differences.

    Args:
        df (pd.DataFrame): The input DataFrame.
        features_list (list): List of column names containing dates.
        reference_date (str, optional): Date used to calculate difference. If None,
            the current date is used. Supports formats like "%Y-%m-%d", "%d-%m-%Y", etc.
        calculate_difference (bool): Whether to compute date differences. Default is True.

    Returns:
        pd.DataFrame: The original DataFrame with new date-related columns.
    """

    df = df.copy()

    for feature in features_list:
        if feature not in df.columns:
            logging.warning("Feature '%s' not in DataFrame", feature)
            continue

        # Vectorized conversion: invalid parses become NaT
        converted = pd.to_datetime(df[feature], errors='coerce')

        # If nothing parsed, skip
        if converted.isna().all():
            logging.warning("No valid dates in feature '%s'; skipping", feature)
            continue

        # Otherwise, replace the column and extract parts
        df[feature] = converted
        df[f'{feature}_year']    = converted.dt.year
        df[f'{feature}_month']   = converted.dt.month
        df[f'{feature}_day']     = converted.dt.day
        df[f'{feature}_weekday'] = converted.dt.day_name()


        if calculate_difference:
            if reference_date is None:
                parsed_reference_date = datetime.today()
            else:
                parsed_reference_date = None
                for fmt in ["%Y-%m-%d", "%d-%m-%Y", "%m/%d/%Y", "%d %b %Y", "%B %d, %Y"]:
                    try:
                        parsed_reference_date = datetime.strptime(reference_date, fmt)
                        break
                    except ValueError:
                        continue
                if parsed_reference_date is None:
                    parsed_reference_date = datetime.today()
                    logging.warning("Unrecognized reference_date format. Using current date.")

            df[f'actual_date - {feature}'] = parsed_reference_date - df[feature]
            df[f'actual_date - {feature} in days'] = (parsed_reference_date - df[feature]).dt.days

    return df


In [9]:
airbnb = create_new_date_columns(airbnb, ['last_review'], calculate_difference=True)